In [1]:
import os

In [2]:
import numpy

In [3]:
from keras.preprocessing import image

Using TensorFlow backend.
C:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\tensor

In [4]:
import cv2

In [5]:
categories = ['with_mask','without_mask']

In [6]:
data = []
for category in categories:
    path = os.path.join('train',category)
    
    label = categories.index(category)

    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)   
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [7]:
len(data)

3059

In [8]:
import random

In [9]:
random.shuffle(data)

In [10]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

In [11]:
len(x)

3059

In [12]:
len(y)

3059

In [13]:
import numpy as np

In [14]:
x = np.array(x)
y = np.array(y)

In [15]:
x.shape

(3059, 224, 224, 3)

In [16]:
y.shape

(3059,)

In [17]:
x = x/255

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [19]:
x_train.shape

(2447, 224, 224, 3)

In [20]:
x_test.shape

(612, 224, 224, 3)

In [21]:
from keras.applications.vgg16 import VGG16

In [22]:
vgg = VGG16()

In [23]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [24]:
from keras import Sequential

In [25]:
model = Sequential()

In [26]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [28]:
for layer in model.layers:
    layer.trainable=False

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [30]:
from keras.layers import Dense

In [31]:
model.add(Dense(1,activation='sigmoid'))

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [33]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [34]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))


Train on 2447 samples, validate on 612 samples
Epoch 1/5
2447/2447 [==============================] - 65s 27ms/step - loss: 0.3709 - accuracy: 0.8611 - val_loss: 0.1791 - val_accuracy: 0.9657
Epoch 2/5
2447/2447 [==============================] - 25s 10ms/step - loss: 0.1713 - accuracy: 0.9485 - val_loss: 0.1169 - val_accuracy: 0.9722
Epoch 3/5
2447/2447 [==============================] - 27s 11ms/step - loss: 0.1360 - accuracy: 0.9600 - val_loss: 0.1367 - val_accuracy: 0.9657
Epoch 4/5
2447/2447 [==============================] - 30s 12ms/step - loss: 0.1082 - accuracy: 0.9665 - val_loss: 0.0754 - val_accuracy: 0.9820
Epoch 5/5
2447/2447 [==============================] - 29s 12ms/step - loss: 0.0900 - accuracy: 0.9751 - val_loss: 0.0686 - val_accuracy: 0.9837


In [39]:
cap = cv2.VideoCapture(0)

In [44]:
while True:
    
    ret, frame = cap.read()
    
    frame = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(frame)
    
    cods = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in cods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    if y_pred == 0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"No Mask",(30,30),(0,0,255))
        
    print(y_pred)
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cap.release()
        
cv2.destroyAllWindows()
    

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
8.916046e-08
9.520691e-06
0.0010136738
7.376819e-05
0.0016633288
0.00088971324
0.0067601358
0.004994233
0.0012047385
0.0060812077
0.0016929149
0.004239272
0.005775075
2.4528144e-06
3.958532e-06
7.919814e-05
9.330378e-06
2.7210592e-05
1.6579721e-05
1.7582139e-06
6.104088e-07
7.190844e-08
2.079196e-07
1.7092722e-08
2.672797e-08
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [43]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    
    return y_pred[0][0]

In [36]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] +2
    end_y = pos[1] + text_size[0][1] -2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),cv2.LINE_AA)

In [50]:
while True:
    
    ret, frame = cap.read()
    
    cods = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in cods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    cv2.imshow("window",frame)
        
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [37]:
cascade_path = 'haarcascade_frontalface_default.xml'
haar = cv2.CascadeClassifier(cascade_path)

if haar.empty():
    print("Error: Unable to load the Haar cascade classifier.")
else:
    print("Haar cascade classifier loaded successfully.")

Haar cascade classifier loaded successfully.


In [38]:
def detect_face(img):
    
    coods = haar.detectMultiScale(img)
    
    return coods